## Getting Started with LangChain on Databricks

### Installing Utilities and Libraries

In [ ]:
%pip install databricks-langchain==0.12.1 langchain-community==0.4.1 langchain-experimental==0.4.1

### Restarting the Python Kernel

In [ ]:
dbutils.library.restartPython()

### Invoking a Chat Model with Databricks

In [ ]:
import json
from databricks_langchain import ChatDatabricks

chat_model = ChatDatabricks(
    endpoint="databricks-claude-sonnet-4-5",
    temperature=0.1,
    max_tokens=250,
)
response = chat_model.invoke("How to use Databricks?")
json_response = response.to_json()
display(json_response['kwargs']['content'])

### Using Mosaic AI Vector Search Index with LangChain

In [ ]:
from databricks_langchain import DatabricksVectorSearch

vector_store = DatabricksVectorSearch(index_name="YOUR_UNITY_CATALOG_NAME.SCHEMA.VECTOR_SEARCH_INDEX_NAME")
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
retriever.invoke("what is the carbonops ESG intelligence Model (CEIM)")

### Creating a Spark Dataframe Agent with LangChain

#### Saving the CSV File in the Databricks File System (DBFS)

In [ ]:
%sh
rm -r /dbfs/delta_lab
mkdir /dbfs/delta_lab
wget -O /dbfs/delta_lab/restaurant_reviews.csv https://raw.githubusercontent.com/kuljotSB/DatabricksGenAIEngineer/refs/heads/main/GenAI_Fundamentals/restaurant_reviews.csv

#### Creating the Spark Dataframe Agent

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_spark_dataframe_agent
from databricks_langchain import ChatDatabricks

df = spark.read.csv("/delta_lab/restaurant_reviews.csv", header=True, inferSchema=True)
display(df)

chat_model = ChatDatabricks(
    endpoint="databricks-claude-sonnet-4-5",
    temperature=0.1,
    max_tokens=250,
)

agent = create_spark_dataframe_agent(llm=chat_model, df=df, verbose=True, allow_dangerous_code=True)


In [ ]:
agent.run("total number of reviews?")

In [ ]:
agent.run("what is the review for cheesecake?")

In [ ]:
agent.run("What is the review given on tacos?")